In [4]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'langfuse',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

In [5]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LANGFUSE_HOST"] = "..."
# os.environ["LANGFUSE_PUBLIC_KEY"] = "..."
# os.environ["LANGFUSE_SECRET_KEY"] = "..."

load_dotenv(Path("../../.env.test"), override=True)

True

In [6]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
from openai import OpenAI
from langfuse import Langfuse

langfuse = Langfuse()
langfuse.auth_check()

openai = OpenAI()

In [8]:
from langfuse.decorators import observe

@observe()
def langchain_chain(inputs):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        ),
        ("user", "{question}")
    ]

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(inputs)
    return answer

def score_answer(answer: str, expected_output: dict):
    """The first argument is the return value from the `langchain_chain` function above."""
    score = int(answer == expected_output.split("#### ")[-1])
    langfuse.score(
        name="correctness",
        value=score,
        trace_id=langfuse.get_trace_id(),
    )
    return {"score": score}

In [9]:
evalset = langfuse.get_dataset("gsm8k-testset")

scores = []
for item in evalset.items:
   answer = langchain_chain(item.input)
   eval = score_answer(answer, item.expected_output)
   scores.append(eval["score"])

print("Average score", sum(scores) / len(scores))

Average score 0.4


In [10]:
from zenbase.adaptors.langfuse_helper import ZenLangfuse

zen_langfuse_helper = ZenLangfuse(langfuse)

In [11]:
from zenbase.core.managers import TraceManager
from zenbase.types import LMRequest, LMDemo, deflm
zenbase_manager = TraceManager()


@zenbase_manager.trace_function
# @retry(
#     stop=stop_after_attempt(3),
#     wait=wait_exponential_jitter(max=8),
#     before_sleep=before_sleep_log(log, logging.WARN),
# )
@observe()
def solver(request: LMRequest):
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_openai import ChatOpenAI

    messages = [
        (
            "system",
            "You are an expert math solver. "
            "You have an question that you should answer, "
            "You have step by step actions that you should take to solve the problem."
            "You have the opertaions that you should do to solve the problem"
            "You should come just with the number for the answer, just the actual number like examples that you have."  # noqa
            ""
            ""
            "Follow the format of the examples as they have the final answer, you need to came up to the plan for solving them.",  # noqa
        )
    ]
    for demo in request.zenbase.task_demos:
        if isinstance(demo.outputs, dict):
            the_output = demo.outputs["answer"]
        else:
            the_output = demo.outputs

        messages += [
            ("user", f'Example Question: {demo.inputs["question"]}'),
            ("assistant", f"Example Answer: {the_output}"),
        ]

    messages.append(("user", "Question: {question}"))
    messages.append(("user", "Plan: {plan}"))
    messages.append(("user", "Mathematical Operation that needed: {operation}"))
    messages.append(
        ("user", "Now come with the answer as number, just return the number, nothing else, just NUMBERS.")
    )

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()

    print("Mathing...")
    plan = planner_chain(request.inputs)
    the_plan = plan["plan"]
    the_operation = operation_finder(
        {
            "plan": the_plan,
            "question": request.inputs["question"],
        }
    )
    inputs_to_answer = {
        "question": request.inputs["question"],
        "plan": the_plan,
        "operation": the_operation["operation"],
    }
    answer = chain.invoke(inputs_to_answer)
    return {"answer": answer}

@zenbase_manager.trace_function
# @retry(
#     stop=stop_after_attempt(3),
#     wait=wait_exponential_jitter(max=8),
#     before_sleep=before_sleep_log(log, logging.WARN),
# )
@observe()
def planner_chain(request: LMRequest):
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_openai import ChatOpenAI

    messages = [
        (
            "system",
            "You are an expert math solver. You have an question that you should create step-by-step plan to solve it. "  # noqa
            "Follow the format of the examples.",
            # noqa
        )
    ]
    if request.zenbase.task_demos:
        for demo in request.zenbase.task_demos[:2]:
            messages += [
                ("user", demo.inputs["question"]),
                ("assistant", demo.outputs["plan"]),
            ]

    messages.append(("user", "{question}"))

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()

    print("Mathing...")
    answer = chain.invoke(request.inputs)
    return {"plan": answer}

@zenbase_manager.trace_function
# @retry(
#     stop=stop_after_attempt(3),
#     wait=wait_exponential_jitter(max=8),
#     before_sleep=before_sleep_log(log, logging.WARN),
# )
@observe()
def operation_finder(request: LMRequest):
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_openai import ChatOpenAI

    messages = [
        (
            "system",
            "You are an expert math solver. You have a plan for solve a problem that is step-by-step, you need to find the overal operation in the math to solve it. "  # noqa
            "Just come up with math operation with simple match operations like sum, multiply, division and minus. "
            ""
            "Follow the format of the examples.",
            # noqa
        )
    ]
    if request.zenbase.task_demos:
        for demo in request.zenbase.task_demos[:2]:
            messages += [
                ("user", demo.inputs["question"]),
                ("user", demo.inputs["plan"]),
                ("assistant", demo.outputs["operation"]),
            ]

    messages.append(("user", "{question}"))
    messages.append(("user", "{plan}"))

    chain = ChatPromptTemplate.from_messages(messages) | ChatOpenAI(model="gpt-3.5-turbo") | StrOutputParser()

    print("Mathing...")
    answer = chain.invoke(request.inputs)
    return {"operation": answer}


def score_answer_with_json(answer: str, demo: LMDemo, langfuse: Langfuse):
    """The first argument is the return value from the `langchain_chain` function above."""
    score = int(answer["answer"] == demo.outputs.split("#### ")[-1])
    langfuse.score(
        name="correctness",
        value=score,
        trace_id=langfuse.get_trace_id(),
    )
    return {"score": score}

In [12]:


from zenbase.optim.metric.bootstrap_few_shot import BootstrapFewShot

train_set = 'GSM8K_train_set_langsmith_dataset_2it3BpoNmwfYa5Nvk6dRButWA56'
validation_set = 'GSM8K_validation_set_langsmith_dataset_2it1pPsf4w75FJ82v5BwweOxnS4'
test_set = 'GSM8K_test_set_langsmith_dataset_2it1pZxHYNfqO8wHTsRP7NiUi1e'
SHOTS = 2
SAMPLES = 2


evaluator_kwargs = dict(
    evaluate=score_answer_with_json,
)


bootstrap_few_shot = BootstrapFewShot(
    shots=SHOTS,
    training_set=train_set,
    test_set=test_set,
    validation_set=validation_set,
    evaluator_kwargs=evaluator_kwargs,
    zen_adaptor=zen_langfuse_helper,
)

teacher_lm, candidates = bootstrap_few_shot.perform(
    solver,
    samples=SAMPLES,
    rounds=1,
    trace_manager=zenbase_manager,
)


Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...
Mathing...


In [14]:
zenbase_manager.all_traces = {}
teacher_lm({"question": "What is 2 + 2?"})

Mathing...
Mathing...
Mathing...


{'answer': '4'}

In [16]:
function_traces = [v for k, v in zenbase_manager.all_traces.items()][0]["optimized"]


In [17]:
from pprint import pprint

pprint(function_traces["planner_chain"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'James writes a 3-page letter to 2 different '
                            'friends twice a week.  How many pages does he '
                            'write a year?'},
        outputs={'plan': 'To find out how many pages James writes in a year, '
                         'we need to multiply the number of pages he writes '
                         'per week by the number of weeks in a year.\n'
                         '\n'
                         '1. Calculate the number of pages James writes per '
                         'week:\n'
                         '   - James writes a 3-page letter to 2 different '
                         'friends twice a week.\n'
                         '   - 3 pages/letter * 2 letters/day = 6 pages/day\n'
                         '   - 6 pages/day * 2 days/week = 12 pages/week\n'
                         '\n'
                         '2. Calculate the number of pages James writes per '
                         'year:\n'
    

In [18]:
from pprint import pprint

pprint(function_traces["operation_finder"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'plan': 'To find out how many pages James writes in a year, we '
                        'need to multiply the number of pages he writes per '
                        'week by the number of weeks in a year.\n'
                        '\n'
                        '1. Calculate the number of pages James writes per '
                        'week:\n'
                        '   - James writes a 3-page letter to 2 different '
                        'friends twice a week.\n'
                        '   - 3 pages/letter * 2 letters/day = 6 pages/day\n'
                        '   - 6 pages/day * 2 days/week = 12 pages/week\n'
                        '\n'
                        '2. Calculate the number of pages James writes per '
                        'year:\n'
                        '   - There are 52 weeks in a year.\n'
                        '   - 12 pages/week * 52 weeks/year = 624 pages/year\n'
                        '\n'
                        'Therefore, James w

In [19]:
from pprint import pprint

pprint(function_traces["solver"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'James writes a 3-page letter to 2 different '
                            'friends twice a week.  How many pages does he '
                            'write a year?'},
        outputs={'answer': '624'},
        original_object=None),
 LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she '
                            'was able to read 12 pages and today, she read '
                            'twice as many pages as yesterday. If she wants to '
                            'read half of the remaining pages tomorrow, how '
                            'many pages should she read?'},
        outputs={'answer': '42'},
        original_object=None),
 LMDemo(inputs={'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money she '
                            'needs. Her parents decided to give her $15 for '
                            'that purpose, and her grandparen